### **Analysing a PPG signal**

---


**PPG:** Photoplethysmogram: 
*   Optically obtained.
*   Plethysmogram is used to detect blood volume changes.

---








First of all, we have to install heartpy database.

In [ ]:
pip install heartpy

In [ ]:
#imports
import heartpy as hp
import matplotlib.pyplot as plt

### **load_exampledata()** :


> Function to load one of the example datasets included in HeartPy.

> ***Parameters:***
* **0 :** a short, very clean PPG signal, sampled at 100.0 Hz
*   **1 :** a slightly longer (~2 minute) PPG signal, with missing signal in first third, and random noise spikes in rest of signal
* **2 :** a long (~11.5 minute) PPG signal recorded 'in the wild' while driving in a driving simulator using a Pulse Sensor on the index finger and an Arduino


> ***Returns:*** (tuple of two arrays) Contains the **data** and **timer** column. If no timer data is available, such as in example 0, an empty second array is returned.











In [ ]:
#loading the clean PPG signal (0)
data, timer = hp.load_exampledata(0)

#visualisation
plt.figure(figsize=(12,4))
plt.plot(data)
plt.show()


### **hp.process()**
> ***Returns:***
* working_data (dict) – dictionary object used to store temporary values.
* measures (dict) – dictionary object used by heartpy to store computed measures.

In [ ]:
wd, m = hp.process(data, sample_rate = 100.0)

### **hp.plotter()**

> Function that uses calculated measures and data stored in the working_data{} and measures{} dict objects to visualise the fitted peak detection solution.

> ***Returns:*** matplotlib plot object.





In [ ]:
#set large figure
plt.figure(figsize=(12,4))

#call plotter
hp.plotter(wd, m)

#display measures computed
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

### **Measures**
>* **BPM:** beats per minute 
* **IBI:** the mean inter-beat interval. 
* **SDNN:** the standard deviation of intervals between heart beats.
* **SDSD:** the standard deviation of successive differences between neighbouring heart beat intervals.
* **RMSSD:** the root mean square of successive differences between neighbouring heart beat intervals.
* **pNN50, pNN20:** the proportion of differences between successive  heart beats greater than 50ms and 20ms.
* **hr_mad:** the median absolute deviation of intervals between heart beats.


In [ ]:
#loading the second dataset included with HeartPy and exploring it:
data, timer = hp.load_exampledata(1)
plt.figure(figsize=(12,4))
plt.plot(data)
plt.show()

There's no signal in the beginning. After the signal commences there's a few noise spikes then the sensor is forcefully moved while recording. This imitates what may happen when recording 'in the wild' as well if the participant moves and accidentally tugs at a sensor cable.

HeartPy is designed to handle this sort of thing out of the box.

In [ ]:
sample_rate = hp.get_samplerate_mstimer(timer)
print('sample rate is: %f Hz' %sample_rate)

The sample_rate was computed from a timer column (that was in ms values). 

This is important beforehand since we didn't know with what sample_rate the signal was sampled. All measures depend on knowing the sample rate.


In [ ]:
wd, m = hp.process(data, sample_rate)

#plot
plt.figure(figsize=(12,4))
hp.plotter(wd, m)

#display measures computed
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

HeartPy comes with two functions: **hp.get_samplerate_mstimer()** that computes sample rate based on a ms-timer, and **hp.get_samplerate_datetime()**, that computes sample_rate based on a column in datetime values.

In [ ]:
#loading the second dataset included with HeartPy and exploring it:
data, timer = hp.load_exampledata(2)
print(timer[0])

When computing the sample rate we need to give get_samplerate_datetime() the format of the string (by default it expects HH:MM:SS.ms):

In [ ]:
sample_rate = hp.get_samplerate_datetime(timer, timeformat='%Y-%m-%d %H:%M:%S.%f')

print('sample rate is: %f Hz' %sample_rate)

In [ ]:
wd, m = hp.process(data, sample_rate, report_time = True)

#plot
plt.figure(figsize=(12,4))
hp.plotter(wd, m)

#let's zoom in on a bit
plt.figure(figsize=(12,4))
plt.xlim(20000, 30000)
hp.plotter(wd, m)

#display measures computed
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

### **Analysing a regular ECG signal**

---


**ECG:** Electrocardiagram: 
*   records the electrical signals in your heart.

---


In [ ]:
#import packages
import heartpy as hp
import matplotlib.pyplot as plt

sample_rate = 250

In [ ]:
data = hp.get_data('e0103.csv')

plt.figure(figsize=(12,4))
plt.plot(data)
plt.show()

In [ ]:
#run analysis
wd, m = hp.process(data, sample_rate)

#visualise in plot of custom size
plt.figure(figsize=(12,4))
hp.plotter(wd, m)

#display computed measures
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

In [ ]:
data = hp.get_data('e0110.csv')

plt.figure(figsize=(12,4))
plt.plot(data)
plt.show()

#and zoom in a bit
plt.figure(figsize=(12,4))
plt.plot(data[0:2500])
plt.show()

We have an issue where the T-wave (the broad wave right after the main QRS complex) is present. We can filter this using a notch filter, as we're interested in the QRS comples.

What the notch filter does is apply a frequency filter to a very narrow frequency range, allowing us to get rid of some things without disturbing the QRS complexes.

In [ ]:
filtered = hp.filter_signal(data, cutoff = 0.05, sample_rate = sample_rate, filtertype='notch')

#visualize again
plt.figure(figsize=(12,4))
plt.plot(filtered)
plt.show()

#and zoom in a bit
plt.figure(figsize=(12,4))
plt.plot(data[0:2500], label = 'original signal')
plt.plot(filtered[0:2500], alpha=0.5, label = 'filtered signal')
plt.legend()
plt.show()

The amplitude of the T-wave is reduced.

In [ ]:
#run analysis
wd, m = hp.process(hp.scale_data(filtered), sample_rate)

#visualise in plot of custom size
plt.figure(figsize=(12,4))
hp.plotter(wd, m)

#display computed measures
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

HeartPy is distrusting some peaks. This is because HeartPy's optimizer likes broader peaks than some ECG recordings provide (especially lower sampling rates). Usually when filtering the peak width decreases as well, potentially causing issues.

We can solve this by upsampling the signal using **scipy.signal.resample**.



In [ ]:
from scipy.signal import resample

#resample the data. Usually 2, 4, or 6 times is enough depending on original sampling rate
resampled_data = resample(filtered, len(filtered) * 2)

#And run the analysis again. Don't forget to up the sample rate as well!
wd, m = hp.process(hp.scale_data(resampled_data), sample_rate * 2)

#visualise in plot of custom size
plt.figure(figsize=(12,4))
hp.plotter(wd, m)

#display computed measures
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

Upsampling the signal has enabled HeartPy to optimize and find the position for all peaks in the signal.

Note: It is recommended to use **hp.scale_data()** in the processing function, when the amplitude is low (2.4-3.8 in the original data).